# Ransomware Deletes Volume Shadow Copies
Detects commands that delete all local volume shadow copies as used by different Ransomware families

## Rule Content
```
- title: Ransomware Deletes Volume Shadow Copies
  id: 4eebe114-4b24-4a9d-9a6c-c7bd7c8eaa61
  status: experimental
  description: Detects commands that delete all local volume shadow copies as used
    by different Ransomware families
  references:
  - https://www.bleepingcomputer.com/news/security/why-everyone-should-disable-vssadmin-exe-now/
  - https://www.hybrid-analysis.com/sample/ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa?environmentId=100
  author: Florian Roth
  date: 2019/06/01
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '*vssadmin delete shadows*'
      - '*wmic SHADOWCOPY DELETE*'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Adminsitrative scripts - e.g. to prepare image for golden image creation
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*vssadmin\ delete\ shadows* OR *wmic\ SHADOWCOPY\ DELETE*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()